In [1]:
%pip install tweepy

  Obtaining dependency information for tweepy from https://files.pythonhosted.org/packages/4d/78/ba0065d5636bbf4a35b78c4f81b74e7858b609cdf69e629d6da5c91b9d92/tweepy-4.14.0-py3-none-any.whl.metadata
  Using cached tweepy-4.14.0-py3-none-any.whl.metadata (3.8 kB)
  Obtaining dependency information for requests<3,>=2.27.0 from https://files.pythonhosted.org/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl.metadata
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Obtaining dependency information for requests-oauthlib<2,>=1.2.0 from https://files.pythonhosted.org/packages/6f/bb/5deac77a9af870143c684ab46a7934038a53eb4aa975bc0687ed6ca2c610/requests_oauthlib-1.3.1-py2.py3-none-any.whl.metadata
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl.metadata (10 kB)
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/12/90/3c9ff0512038035f59d279fddeb79f5f1eccd885


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/ed/8c/87ddf1fcb55d11f9f847e3c69bb1c6f8e46e2f40ab1a2d2abadb2401b007/pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/11/c3/005fcca25ce078d2cc29fd559379817424e94885510568bc1bc53d7d5846/pytz-2024.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/a6/ab/7e5f53c3b9d14972843a647d8d7a853969a58aecc7559cb3267302c94774/tzdata-2024.2-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.6 MB 4.2 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/11.6 MB 12.1 MB/s eta 0:00:01
   ---- ----------------------------------- 1.4/11.6 MB 12.8 MB/s eta 0:00:01
   -------- ------------------------------- 2.4/11.6 MB 13.8 MB/s eta 0:00:

In [5]:
import tweepy
import pandas as pd
import json
from datetime import datetime, timedelta

# Load credentials from the .json file
with open("twitter_cred.json", "r") as file:
    creds = json.load(file)

# Function to authenticate with a given account's credentials
def authenticate(account_name):
    auth = tweepy.OAuth1UserHandler(
        creds[account_name]["API_KEY"],
        creds[account_name]["API_SECRET"],
        creds[account_name]["ACCESS_TOKEN"],
        creds[account_name]["ACCESS_TOKEN_SECRET"]
    )
    return tweepy.API(auth)

# Function to fetch popular tweets for a given query
def fetch_popular_tweets(api, query, max_tweets=100):  # Limit to 100 tweets per account
    tweets = []
    try:
        for tweet in tweepy.Cursor(
            api.search_tweets,
            q=query,
            lang="en",  # Restrict to English
            result_type="popular"  # Focus on tweets with high engagement
        ).items(max_tweets):
            tweets.append({
                "created_at": tweet.created_at,
                "text": tweet.text,
                "user": tweet.user.screen_name,
                "retweets": tweet.retweet_count,
                "likes": tweet.favorite_count,
                "id": tweet.id
            })
    except tweepy.errors.TweepyException as e:
        print(f"Error fetching tweets: {e}")
    return tweets

# Function to handle date assignments to accounts
def assign_dates_to_accounts(start_date, num_days_per_account):
    date_assignments = {}
    # Distribute 3 consecutive days across each of 5 accounts
    for i in range(5):  # 5 accounts
        account_name = f"account{i+1}"
        assigned_dates = []
        for j in range(num_days_per_account):  # Each account gets 3 days
            assigned_dates.append(start_date + timedelta(days=i * num_days_per_account + j))
        date_assignments[account_name] = assigned_dates
    return date_assignments

# Main Script
start_date = datetime(2023, 2, 10)  # Set to the game release date (Feb 10, 2023)
num_days_per_account = 3  # Each account gets 3 days of data

# Assign specific dates to each account
date_assignments = assign_dates_to_accounts(start_date, num_days_per_account)

# Loop through each account and fetch data for the assigned dates
tweets_all = []
for account_name, dates in date_assignments.items():
    # Authenticate with the current account
    api = authenticate(account_name)
    
    for date in dates:
        # Prepare the query for the given date
        query = f'"Hogwarts Legacy" OR #HogwartsLegacy lang:en since:{date.date()} until:{(date + timedelta(days=1)).date()}'
        print(f"Fetching tweets for {account_name} on {date.date()}")
        
        # Fetch tweets for that day (up to the 100 tweet limit per account for the month)
        tweets = fetch_popular_tweets(api, query, max_tweets=100)  # Limit to 100 tweets per account
        tweets_all.extend(tweets)

# Save all collected tweets to a CSV file
df = pd.DataFrame(tweets_all)
df.to_csv("hogwarts_legacy_popular_tweets.csv", index=False)
print(f"Saved {len(tweets_all)} popular tweets to 'hogwarts_legacy_popular_tweets.csv'.")


Fetching tweets for account1 on 2023-02-10
Error fetching tweets: 403 Forbidden
453 - You currently have access to a subset of X API V2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.x.com/en/portal/product
Fetching tweets for account1 on 2023-02-11
Error fetching tweets: 403 Forbidden
453 - You currently have access to a subset of X API V2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.x.com/en/portal/product
Fetching tweets for account1 on 2023-02-12
Error fetching tweets: 403 Forbidden
453 - You currently have access to a subset of X API V2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: h